In [1]:
import requests
import pandas as pd
import numpy as np
from pprint import pprint
# import config
import re

In [2]:
# Import json and convert to DF
url = 'https://www.thecocktaildb.com/api/json/v2/9973533/search.php?s'

response = requests.get(url).json()

drinks_df = pd.json_normalize(response['drinks'], max_level=1)
drinks_df.head()

,idDrink,strDrink,strDrinkAlternate,strTags,strVideo,strCategory,strIBA,strAlcoholic,strGlass,strInstructions,...,strMeasure10,strMeasure11,strMeasure12,strMeasure13,strMeasure14,strMeasure15,strImageSource,strImageAttribution,strCreativeCommonsConfirmed,dateModified
0,15997,GG,None,None,None,Ordinary Drink,None,Optional alcohol,Collins Glass,Pour the Galliano liqueur over ice. Fill the r...,...,None,None,None,None,None,None,None,None,No,2016-07-18 22:06:00
1,17222,A1,None,None,None,Cocktail,None,Alcoholic,Cocktail glass,"Pour all ingredients into a cocktail shaker, m...",...,None,None,None,None,None,None,None,None,No,2017-09-07 21:42:09
2,13501,ABC,None,None,None,Shot,None,Alcoholic,Shot glass,Layered in a shot glass.,...,None,None,None,None,None,None,None,None,No,2016-08-31 19:32:08
3,17203,Kir,None,"IBA,ContemporaryClassic",None,Ordinary Drink,None,Alcoholic,Wine Glass,Add the crème de cassis to the bottom of the g...,...,None,None,None,None,None,None,None,None,No,2017-09-02 17:38:14
4,14229,747,None,None,None,Shot,None,Alcoholic,Shot glass,"pour kaluha, then Baileys, then Frangelico not...",...,None,None,None,None,None,None,None,None,No,2016-08-31 19:28:26


In [ ]:
# drinks_df.columns

In [12]:
measure_df = drinks_df[['idDrink', 'strDrink',  'strIngredient1', 'strMeasure1', 'strIngredient2', 'strMeasure2',
                        'strIngredient3', 'strMeasure3', 'strIngredient4', 'strMeasure4','strIngredient5', 'strMeasure5',
                        'strIngredient6', 'strMeasure6', 'strIngredient7', 'strMeasure7','strIngredient8', 'strMeasure8',
                        'strIngredient9', 'strMeasure9', 'strIngredient10', 'strMeasure10', 'strIngredient11', 'strMeasure11',
                        'strIngredient12', 'strMeasure12']]

measure_df
# measure_df.to_excel("measure_df.xlsx")


In [42]:
measurements = []
units = []

# check if numeric or alpha 
for index, row in measure_df.iterrows():
    string_list = row['strMeasure1']
    if (string_list):
            wordsRegex = "[a-zA-z]+"
            numbersRegex = "(\d [\d\/\d]*)"
            words = re.search(wordsRegex, string_list)
            measure = re.search(numbersRegex, string_list)
            if (words):
                units.append(words.group(0))
            else:
                units.append("")
            if (measure):
                measurements.append(measure.group(0))
            else:
                measurements.append("")
    else:
        measurements.append("")
        units.append("")
                
measurements_df = pd.DataFrame(
    {"Measurements": measurements, "Units": units}, 
)
measurements_df.head()

,Measurements,Units
0,2 1/2,shots
1,1 3/4,shot
2,3,
3,1,part
4,3,part


In [41]:
measurements_df["Units"].unique()

array([''], dtype=object)

In [8]:
# split measurements column on the space, rename columns 
split_measurements_df = pd.DataFrame(measurements_df['Measurements'].str.split(' ',1).tolist(),
                                 columns = ['Measurement_a','Measurement_b'])

# Potential fractions to be dealt with: ['1/2', '3/4', '2/3', '1/4']
# replace with decimals
split_measurements_df['Measurement_b'] = split_measurements_df['Measurement_b'].replace(['1/2','3/4','2/3','1/4'],
                                                                                        [.5,.75,.66,.25])
# convert string to number
split_measurements_df['Measurement_a'] = pd.to_numeric(split_measurements_df['Measurement_a'], errors='coerce')
split_measurements_df['Measurement_b'] = pd.to_numeric(split_measurements_df['Measurement_b'], errors='coerce')

# make all NANs = 0
split_measurements_df['Measurement_b'] = split_measurements_df['Measurement_b'].fillna(0)

# create a new column with total
split_measurements_df['Total'] = split_measurements_df['Measurement_a'] + split_measurements_df['Measurement_b']

# add back the units column
split_measurements_df['Units'] = units

split_measurements_df.head()

,Measurement_a,Measurement_b,Total,Units
0,2.0,0.50,2.50,shots
1,1.0,0.75,1.75,shot
2,3.0,0.00,3.00,
3,1.0,0.00,1.00,part
4,3.0,0.00,3.00,part


In [43]:
all_units = ['shots', 'shot', '', 'part', 'oz', 'cl', 'dash', 'pint', 'parts',
       'Chilled', 'tsp', 'Juice', 'cup', 'bottle', 'dashes', 'ml',
       'tblsp', 'L', 'mikey', 'jiggers', 'fifth', 'fresh', 'cups',
       'pieces', 'can', 'scoops', 'handful', 'long', 'cL', 'bottles',
       'qt', 'dl', 'gr', 'quart', 'measures', 'glass', 'cubes', 'jigger',
       'black', 'full', 'kg', 'inch', 'lb', 'large', '', 'Shot', 'parts', 'part', 'shot', 'oz', 'cl', 'dashes',
       'Unsweetened', 'splash', 'pint', 'Juice', 'shots', 'tsp', 'tblsp',
       'cup', 'Add', 'dash', 'ml', 'large', 'jigger', 'Squeeze', 'frozen',
       'Turkish', 'L', 'fifth', 'whole', 'cups', 'gal', 'gr', 'packages',
       'bottle', 'can', 'very', 'cans', 'tbsp', 'Top', 'drops', 'quart',
       'orange', 'glass', 'About', 'slice', 'piece', 'Strong', 'scoops',
       'chunks','', 'Shot', 'part', 'shot', 'tsp', 'oz', 'Fresh', 'Fill', 'cl',
       'dash', 'parts', 'splash', 'dashes', 'Juice', 'seltzer', 'cup',
       'cubes', 'tblsp', 'to', 'slice', 'cube', 'twist', 'gr', 'ml',
       'Twist', 'shots', 'Top', 'drops', 'whole', 'wedge', 'cups',
       'frozen', 'if', 'wedges', 'L', 'lb', 'chunk', 'can', 'scoops',
       'full', 'Chilled', 'gal', 'splashes', 'fifth', 'tbsp', 'Garnish',
       'By', 'pint', 'glass', 'pods', 'cracked', 'Dash', 'large',
       'spoons', 'stick', 'ripe', 'A', 'Full', 'Cup', 'inch','', 'Shot', 'shot', 'part', 'tsp', 'oz', 'slice', 'dashes', 'cup',
       'top', 'cl', 'cubes', 'ml', 'twist', 'or', 'dash', 'Twist',
       'Float', 'pinches', 'Fresh', 'Top', 'Dash', 'chunks', 'Bacardi',
       'dl', 'L', 'Juice', 'whole', 'piece', 'pinch', 'cups', 'crushed',
       'tblsp', 'drops', 'splash', 'About', 'lots', 'package', 'parts',
       'Chopped', 'jiggers', 'Sprig', 'gal', 'Fill', 'glass', 'Claret',
       'scoops', 'wedge', 'Garnish', 'Coarse', 'qt', 'pint', 'Add',
       'Whole', 'one', 'beaten', 'large', 'inch', 'for','', 'Fresh', 'tsp', 'oz', 'fill', 'dash', 'Dash', 'cl', 'twist',
       'Twist', 'slice', 'crushed', 'top', 'Around', 'cubes', 'Top',
       'cup', 'small', 'Garnish', 'drops', 'ground', 'tblsp', 'Over',
       'parts', 'Chopped', 'dashes', 'gal', 'garnish', 'wedges', 'to',
       'cups', 'Wedges', 'part', 'Juice', 'pint', 'qt', 'splash',
       'Dashes', 'whole', 'large', 'Ground', 'shot', 'Grated', 'mini',
       'tablespoons', 'inch','', 'tsp', 'twist', 'pinch', 'Garnish', 'tblsp', 'dashes', 'Dash',
       'pint', 'wedge', 'slice', 'piece', 'ml', 'parts', 'Top', 'L',
       'cup', 'wedges', 'long', 'oz', 'Slice', 'cubes', 'splash', 'fifth',
       'Pinch', 'cl', 'Rimmed', 'Twist', 'whole', 'Fresh', 'shots',
       'bottle', 'shot', 'Fill', 'tablespoons','', 'drop', 'Fill', 'pint', 'tsp', 'whole', 'cup', 'part', 'oz',
       'Large', 'dash', 'sprigs', 'tblsp', 'cups', 'Top', 'Garnish',
       'shot','', 'cup', 'to', 'oz', 'pint', 'crushed', 'drops', 'piece',
       'sticks', 'wedge','', 'wedge', 'drops', 'cup', 'Garnish','', 'cup', 'drops','', 'Ground', 'tsp', 'drop','', 'drop']

In [46]:
all_units_unique = list(set(all_units))
all_units_unique

['',
 'frozen',
 'cL',
 'Fresh',
 'long',
 'Large',
 'large',
 'A',
 'Chopped',
 'qt',
 'shot',
 'spoons',
 'Rimmed',
 'Dash',
 'Cup',
 'cans',
 'quart',
 'drop',
 'whole',
 'stick',
 'for',
 'fresh',
 'L',
 'Wedges',
 'Strong',
 'Dashes',
 'Unsweetened',
 'sticks',
 'pieces',
 'jigger',
 'ground',
 'Garnish',
 'ripe',
 'Coarse',
 'or',
 'bottle',
 'Turkish',
 'chunk',
 'fifth',
 'top',
 'part',
 'Fill',
 'shots',
 'beaten',
 'tablespoons',
 'Grated',
 'inch',
 'pinch',
 'Whole',
 'lots',
 'crushed',
 'to',
 'Over',
 'very',
 'measures',
 'Add',
 'gr',
 'small',
 'Chilled',
 'tbsp',
 'handful',
 'About',
 'cracked',
 'one',
 'pinches',
 'kg',
 'tsp',
 'cubes',
 'cup',
 'Ground',
 'tblsp',
 'splash',
 'Sprig',
 'dl',
 'Twist',
 'wedges',
 'can',
 'pint',
 'pods',
 'parts',
 'full',
 'Shot',
 'slice',
 'Full',
 'Bacardi',
 'sprigs',
 'jiggers',
 'drops',
 'Pinch',
 'cube',
 'oz',
 'By',
 'glass',
 'Claret',
 'splashes',
 'Around',
 'black',
 'cl',
 'wedge',
 'Slice',
 'Float',
 'mini',
 

In [48]:
actual_units = ['','cL','qt','shot','spoons','Dash','Cup','cans','quart','drop','L','Dashes','jigger','bottle',
                'fifth','top','part','Fill','shots','beaten','tablespoons','inch','gr','tbsp','handful','pinches',
                'kg','tsp','cubes','cup','tblsp','splash','dl','can','pint','parts','full','Shot','Full','Bacardi',
                'jiggers','drops','Pinch','cube','oz','splashes','cl','Float','lb','ml','bottles','scoops','Squeeze',
                'fill','package','packages','dash','dashes','gal','seltzer','Juice','cups','Top']

In [49]:
measurement_conversions = {'unit': actual_units,
                             'multiplier': [0,0.338,32,1.5,0.5,0.25,8,12,32,0.01,33.8,0.25,1.5,12,
                                           25.5,1,1.5,1,1.5,2,0.5,1,0.035,0.5,1,0.15,
                                           35.274,0.167,0.5,8,0.5,0.5,3.38,12,16,1.5,1,1.5,1,1.5,
                                           1.5,0.01,0.15,0.5,1,0.5,0.338,1,16,0.0338,12,2,0.25,
                                           1,1,1,0.25,0.25,124,1,1,8,1]}
measurement_conversions_df = pd.DataFrame.from_dict(measurement_conversions)

measurement_conversions_df.head(10)

,unit,multiplier
0,,0.000
1,cL,0.338
2,qt,32.000
3,shot,1.500
4,spoons,0.500
5,Dash,0.250
6,Cup,8.000
7,cans,12.000
8,quart,32.000
9,drop,0.010


In [50]:
def units_to_oz(row):
    unit = row["Units"]
    #if unit in measurement_conversions_df, then
    mult = measurement_conversions_df.loc[measurement_conversions_df["unit"] == unit, "multiplier"].item()
    t = row["Total"]
    row["Total"] = t * mult
    #else
    #row["Total"] = 0

    return row

In [51]:
temp = split_measurements_df.apply(units_to_oz, axis = 1)
temp.head()

ValueError: can only convert an array of size 1 to a Python scalar

In [5]:
count = 0

def split(df_column):
    global count
    print("count:", count)
    count += 1
    print(type(df_column))
#     df_column = str(df_column)

    ### this section separates numbers from words
    measurements = []
    units = []
    
    try: 
        for index, string_list in df_column.items():
            #string_list = row['strMeasure1']
            if (string_list):
                    wordsRegex = "[a-zA-z]+"
                    numbersRegex = "(\d [\d\/\d]*)"
                    words = re.search(wordsRegex, string_list)
                    measure = re.search(numbersRegex, string_list)
                    if (words):
                        units.append(words.group(0))
                    else:
                        units.append("")
                    if (measure):
                        measurements.append(measure.group(0))
                    else:
                        measurements.append("")
            else:
                measurements.append("")
                units.append("")

        measurements_df = pd.DataFrame(
            {"Measurements": measurements, "Units": units}, 
        )
        print(measurements_df)

        ### this section converts the fractions to floats

        # split measurements column on the space, rename columns 
        split_measurements_df = pd.DataFrame(measurements_df['Measurements'].str.split(' ',1).tolist(),
                                         columns = ['Measurement_a','Measurement_b'])

        # Potential fractions to be dealt with: ['1/2', '3/4', '2/3', '1/4']
        # replace with decimals
        split_measurements_df['Measurement_b'] = split_measurements_df['Measurement_b'].replace(['1/2','3/4','2/3','1/4'],
                                                                                                [.5,.75,.66,.25])
        # convert string to number
        split_measurements_df['Measurement_a'] = pd.to_numeric(split_measurements_df['Measurement_a'], errors='coerce')
        split_measurements_df['Measurement_b'] = pd.to_numeric(split_measurements_df['Measurement_b'], errors='coerce')

        # make all NANs = 0
        split_measurements_df['Measurement_b'] = split_measurements_df['Measurement_b'].fillna(0)

        # create a new column with total
        split_measurements_df['Total'] = split_measurements_df['Measurement_a'] + split_measurements_df['Measurement_b']

        # add back the units column
        split_measurements_df['Units'] = units
        #split_measurements_df

        ### this section converts to ounces using pre-defined multipliers
        # use units_to_oz with apply function
        split_measurements_df['Total'] = split_measurements_df.apply(units_to_oz, axis=1)

        df_column = split_measurements_df["Total"]
    
    except:
        pass
    
    return df_column # this returns a series



In [6]:
# new_df = measure_df[['strMeasure1', 'strMeasure2',
#        'strMeasure3', 'strMeasure4', 'strMeasure5', 'strMeasure6',
#        'strMeasure7', 'strMeasure8', 'strMeasure9', 'strMeasure10',
#        'strMeasure11', 'strMeasure12', 'strMeasure13', 'strMeasure14',
#        'strMeasure15']].apply(split, axis = 1)
new_df = measure_df['strMeasure1'].apply(split)
new_df

count: 0
<class 'str'>
count: 1
<class 'str'>
count: 2
<class 'str'>
count: 3
<class 'str'>
count: 4
<class 'str'>
count: 5
<class 'str'>
count: 6
<class 'str'>
count: 7
<class 'str'>
count: 8
<class 'str'>
count: 9
<class 'str'>
count: 10
<class 'str'>
count: 11
<class 'str'>
count: 12
<class 'str'>
count: 13
<class 'str'>
count: 14
<class 'str'>
count: 15
<class 'str'>
count: 16
<class 'str'>
count: 17
<class 'str'>
count: 18
<class 'NoneType'>
count: 19
<class 'str'>
count: 20
<class 'str'>
count: 21
<class 'str'>
count: 22
<class 'str'>
count: 23
<class 'str'>
count: 24
<class 'str'>
count: 25
<class 'str'>
count: 26
<class 'str'>
count: 27
<class 'str'>
count: 28
<class 'str'>
count: 29
<class 'str'>
count: 30
<class 'str'>
count: 31
<class 'NoneType'>
count: 32
<class 'str'>
count: 33
<class 'str'>
count: 34
<class 'str'>
count: 35
<class 'str'>
count: 36
<class 'str'>
count: 37
<class 'str'>
count: 38
<class 'str'>
count: 39
<class 'str'>
count: 40
<class 'str'>
count: 41
<class

count: 481
<class 'str'>
count: 482
<class 'str'>
count: 483
<class 'str'>
count: 484
<class 'str'>
count: 485
<class 'str'>
count: 486
<class 'str'>
count: 487
<class 'str'>
count: 488
<class 'str'>
count: 489
<class 'str'>
count: 490
<class 'str'>
count: 491
<class 'str'>
count: 492
<class 'str'>
count: 493
<class 'str'>
count: 494
<class 'str'>
count: 495
<class 'str'>
count: 496
<class 'str'>
count: 497
<class 'str'>
count: 498
<class 'str'>
count: 499
<class 'str'>
count: 500
<class 'str'>
count: 501
<class 'str'>
count: 502
<class 'str'>
count: 503
<class 'str'>
count: 504
<class 'str'>
count: 505
<class 'str'>
count: 506
<class 'str'>
count: 507
<class 'str'>
count: 508
<class 'str'>
count: 509
<class 'str'>
count: 510
<class 'str'>
count: 511
<class 'str'>
count: 512
<class 'str'>
count: 513
<class 'str'>
count: 514
<class 'str'>
count: 515
<class 'str'>
count: 516
<class 'str'>
count: 517
<class 'str'>
count: 518
<class 'str'>
count: 519
<class 'str'>
count: 520
<class 'str'>


0        2 1/2 shots 
1         1 3/4 shot 
2                1/3 
3             1 part 
4           1/3 part 
            ...      
613             2 oz 
614    1/2 lb frozen 
615             1 oz 
616       1 oz white 
617              2 oz
Name: strMeasure1, Length: 618, dtype: object

In [ ]:
test_df = measure_df.copy(deep=True)
test_df.head()

In [ ]:
# def eda_helper(df):
#     dict_list = []
#     for col in df.columns:
#         data = df[col]
#         dict_ = {}
#         # The null count for a column. Columns with no nulls are generally more interesting
#         dict_.update({"null_count" : data.isnull().sum()})
#         # Counting the unique values in a column
#         # This is useful for seeing how interesting the column might be as a feature
#         dict_.update({"unique_count" : len(data.unique())})
#         # Finding the types of data in the column
#         # This is useful for finding out potential problems with a column having strings and ints
#         dict_.update({"data_type" : str(set([type(d).__name__ for d in data]))})
#         #dict_.update({"score" : match[1]})
        
#         # Check which columns have duplicated values 
#         dict_.update({"duplicates" : data.duplicated().any()})
#         dict_list.append(dict_)
#     eda_df = pd.DataFrame(dict_list)
#     eda_df.index = df.columns
        
#     return eda_df
# eda_helper(test_df)

In [ ]:
# cols = ['strMeasure1', 'strMeasure2',
#        'strMeasure3', 'strMeasure4', 'strMeasure5', 'strMeasure6',
#        'strMeasure7', 'strMeasure8', 'strMeasure9', 'strMeasure10',
#        'strMeasure11', 'strMeasure12', 'strMeasure13', 'strMeasure14',
#        'strMeasure15']

# # work on one column at a time to replace measure with a total in OZ. 
# for col in cols:
    
#     # use preprocessing with transform function
#     test_df[col] = test_df[col].transform(split) # axis=1
#     print(test_df[col])

In [ ]:
# test_df.head()

In [ ]:
# measurements_df["Units"].unique()

In [ ]:
mult = measurement_conversions_df.loc[measurement_conversions_df["unit"] == "shots "]["multiplier"]
mult